In [11]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

#Frame per second to slow down the Video
fps = 25

#Path of the video file
path_to_file = 'solidYellowLeft.mp4'

# Connects to our Video File
cap = cv2.VideoCapture(path_to_file)

# Automatically grab width and height from video feed
# (returns float which we need to convert to integer for later on!)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Writer object which will be used to finally create our outputfile
writer = cv2.VideoWriter('solid_yellow_left_output.mp4', cv2.VideoWriter_fourcc(*'VIDX'), 25, (width, height))

while cap.isOpened():
    
    # Read in and grayscale the frame from the video
    _, frame = cap.read()
    
    if _ == True:
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Define a kernel size and apply Gaussian smoothing
        kernel_size = 9
        blurr_frame = cv2.GaussianBlur(gray_frame, (kernel_size, kernel_size), 0)

        # Define our parameters for Canny and apply
        low_threshold = 100
        high_threshold = 200

        edge_frame = cv2.Canny(blurr_frame, low_threshold, high_threshold)

        #Taking the copy of canny frame
        edge_copy = edge_frame.copy()


        # Next we'll create a masked edges frame using cv2.fillPoly()
        mask = np.zeros_like(edge_frame)   
        ignore_mask_color = 255   

        # This time we are defining a four sided polygon to mask
        imshape = frame.shape
        vertices = np.array([[(50,imshape[0]),(450, 275), (500, 290), (900,imshape[0])]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)

        masked_edges = cv2.bitwise_and(edge_frame, mask)

        # Define the Hough transform parameters
        # Make a blank the same size as our image to draw on

        rho = 1 # distance resolution in pixels of the Hough grid

        theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 65   # minimum number of votes (intersections in Hough grid cell)

        min_line_length = 40 #minimum number of pixels making up a line
        max_line_gap = 5    # maximum gap in pixels between connectable line segments

        #creating a blank to draw lines on
        zero_frame = frame.copy()*0 

        # Run Hough on edge detected image
        lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)


        # Iterate over the output "lines" and draw lines on the blank
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(zero_frame, (x1,y1), (x2,y2), (0 , 255, 255), 4)
                cv2.line(frame, (x1, y1), (x2, y2), (255, 255, 0), 4)


        # Create a "color" binary frame to combine with line image
        color_edges = np.dstack((edge_copy, edge_copy, edge_copy)) 

        # Draw the lines on the edge image
        combo = cv2.addWeighted(color_edges, 0.8, zero_frame, 1, 0) 
        
        #Writing our output in 'solid_yellow_left_output.mp4' file
#         writer.write(frame)
        
        time.sleep(1/fps)
        cv2.imshow('Lane Detection', frame)

        k = cv2.waitKey(1) & 0xFF

        if  k == 13:
            break
            
    else:
        
        break
        
#Releasing and Destroying all Opened Windows
cap.release()
writer.release()
cv2.destroyAllWindows()
